# SSL 连接示例

## 通过 SSL 连接到 Redis 实例

In [ ]:
import redis

r = redis.Redis(
    host='localhost', 
    port=6666, 
    ssl=True, 
    ssl_cert_reqs="none",
)
r.ping()

True

## 通过 URL 字符串连接到 Redis 实例

In [ ]:
import redis

r = redis.from_url("rediss://localhost:6666?ssl_cert_reqs=none&decode_responses=True&health_check_interval=2")
r.ping()

True

## 使用 ConnectionPool 连接到 Redis 实例

In [ ]:
import redis

redis_pool = redis.ConnectionPool(
    host="localhost", 
    port=6666, 
    connection_class=redis.SSLConnection, 
    ssl_cert_reqs="none",
)

r = redis.StrictRedis(connection_pool=redis_pool) 
r.ping()

True

## 通过 SSL 连接到 Redis 实例，同时指定最低 TLS 版本

In [ ]:
import redis
import ssl

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_min_version=ssl.TLSVersion.TLSv1_3,
    ssl_cert_reqs="none",
)
r.ping()

True

## 通过 SSL 连接到 Redis 实例，同时指定自签名的 SSL CA 证书

In [ ]:
import os
import redis

pki_dir = os.path.join("..", "..", "dockers", "stunnel", "keys")

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_certfile=os.path.join(pki_dir, "client-cert.pem"),
    ssl_keyfile=os.path.join(pki_dir, "client-key.pem"),
    ssl_cert_reqs="required",
    ssl_ca_certs=os.path.join(pki_dir, "ca-cert.pem"),
)
r.ping()

True

## 通过 SSL 连接到 Redis 实例，并验证证书的 OCSP 状态

redis 包旨在保持小巧，这意味着必须安装额外的库以支持 OCSP stapling。因此，首先通过以下命令安装 redis：

`pip install redis[ocsp]`

这将安装 cryptography、requests 和 PyOpenSSL，这些库通常不需要用来使用 Redis。

在下一个示例中，我们将通过 SSL 连接到 Redis 实例，并验证证书的 OCSP 状态。然而，我们使用的证书没有 AIA 扩展，这意味着无法执行 OCSP 验证。

In [ ]:
import os
import redis

pki_dir = os.path.join("..", "..", "dockers", "stunnel", "keys")

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_certfile=os.path.join(pki_dir, "client-cert.pem"),
    ssl_keyfile=os.path.join(pki_dir, "client-key.pem"),
    ssl_cert_reqs="required",
    ssl_ca_certs=os.path.join(pki_dir, "ca-cert.pem"),
    ssl_validate_ocsp=True,
)

try:
    r.ping()
except redis.ConnectionError as e:
    assert e.args[0] == "No AIA information present in ssl certificate"
    print("OCSP validation failed as expected.")

OCSP validation failed as expected.


## 通过 SSL 连接到 Redis 实例，并验证 OCSP stapled 证书

也可以验证 OCSP stapled 响应。同样，在这个示例中，服务器并未发送 OCSP stapled 响应，因此验证将失败。

In [ ]:
import os
import redis

pki_dir = os.path.join("..", "..", "dockers", "stunnel", "keys")
ca_cert = os.path.join(pki_dir, "ca-cert.pem")

# It is possible to specify an expected certificate, or leave it out.
expected_certificate = open(ca_cert, 'rb').read()

# If needed, a custom SSL context for OCSP can be specified via ssl_ocsp_context

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_certfile=os.path.join(pki_dir, "client-cert.pem"),
    ssl_keyfile=os.path.join(pki_dir, "client-key.pem"),
    ssl_cert_reqs="required",
    ssl_ca_certs=ca_cert,
    ssl_validate_ocsp_stapled=True,
    ssl_ocsp_expected_cert=expected_certificate,
)

try:
    r.ping()
except redis.ConnectionError as e:
    assert e.args[0] == "no ocsp response present"
    print("OCSP validation failed as expected.")

OCSP validation failed as expected.
